## Lesson 6 

In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using Theano backend.


In [2]:
#downloading the data
path = get_file('nietzsche.txt', origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt")
text = open(path).read()
print('corpus length:', len(text))

corpus length: 600901


In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)+1
print("total chars: ", vocab_size)

total chars:  86


In [4]:
chars.insert(0, "\0")

In [5]:
''.join(chars[1:-6]) #2nd element to 7th from the back.

'\n !"\'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz'

In [6]:
char_indices = dict((c,i) for i,c in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))

In [7]:
idx = [char_indices[c] for c in text]

In [8]:
idx[:10]

[40, 42, 29, 30, 25, 27, 29, 1, 1, 1]

In [9]:
''.join(indices_char[i] for i in idx[:70])

'PREFACE\n\n\nSUPPOSING that Truth is a woman--what then? Is there not gro'

In [10]:
len(text)

600901

#### 3 Char model
create inputs 

In [11]:
cs = 3 
c1_dat = [idx[i] for i in xrange(0,len(idx)-1-cs, cs)]
c2_dat = [idx[i+1] for i in xrange(0,len(idx)-1-cs, cs)]
c3_dat = [idx[i+2] for i in xrange(0,len(idx)-1-cs, cs)]
c4_dat = [idx[i+3] for i in xrange(0,len(idx)-1-cs, cs)]



In [12]:
## define inputs
x1 = np.stack(c1_dat[:-2])
x2 = np.stack(c2_dat[:-2])
x3 = np.stack(c3_dat[:-2])

In [13]:
## define output
y = np.stack(c4_dat[:-2])

Basically you are trying to use the first three sequence to predict the forth:

In [14]:
print(idx[:4])
print(x1[0])
print(x2[0])
print(x3[0])
print(y[0])

[40, 42, 29, 30]
40
42
29
30


---
## Modeling start 
---
handed coded RNN

In [15]:
#number of lanet factors:
n_fac = 42 

In [16]:
#create input and embedding outputs
def embedding_input(name, n_in,n_out):
    inp = Input(shape=(1,), dtype = 'int64', name = name)
    emb = Embedding(n_in, n_out, input_length=1)(inp)
    return inp, Flatten()(emb)

In [17]:
c1_in, c1 = embedding_input('c1', vocab_size, n_fac)
c2_in, c2 = embedding_input('c2', vocab_size, n_fac)
c3_in, c3 = embedding_input('c3', vocab_size, n_fac)

## Create and train model

In [18]:
n_hidden = 256
#first layer - operation from input to hidden
dense_in = Dense(n_hidden, activation='relu')


In [19]:
#first hidden activation is applied to the result of the embedding of the first character 
c1_hidden = dense_in(c1)

In [20]:
dense_hidden = Dense(n_hidden, activation='tanh')


In [21]:
#Our second and third hidden activations sum up the previous hidden state (after applying dense_hidden) to the new input state.


In [22]:
c2_dense = dense_in(c2)
hidden_2 = dense_hidden(c1_hidden)
c2_hidden = merge([c2_dense, hidden_2])

c3_dense = dense_in(c3)
hidden_3 = dense_hidden(c2_hidden)
c3_hidden = merge([c3_dense, hidden_3])

dense_out = Dense(vocab_size, activation='softmax')

c4_out = dense_out(c3_hidden)


In [23]:
model = Model([c1_in, c2_in, c3_in], c4_out)


In [24]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())


In [25]:
model.optimizer.lr=0.001
model.fit([x1, x2, x3], y, batch_size=64, nb_epoch=4, verbose= 2)


Epoch 1/4
10s - loss: 2.4029
Epoch 2/4
10s - loss: 2.2623
Epoch 3/4
11s - loss: 2.2171
Epoch 4/4
11s - loss: 2.1751


In [26]:
def get_next(inp):
    idxs = [char_indices[c] for c in inp] #convert character to index
    arrs = [np.array(i)[np.newaxis] for i in idxs] #add a new dimension to each label
    p = model.predict(arrs)
    i = np.argmax(p)
    
    return chars[i] #return back the text

In [27]:
get_next('phi')

'l'

In [28]:
get_next(" th")

'e'

In [29]:
get_next(" an")

'd'

---
## Second RNN
---
Arbitary deep, in this case 8.



In [30]:
cs=8

c_in_dat = [[idx[i+n] for i in xrange(0, len(idx)-1-cs, cs)] for n in range(cs)] 

c_out_dat = [idx[i+cs] for i in xrange(0, len(idx)-1-cs, cs)]

xs = [np.stack(c[:-2]) for c in c_in_dat]


In [31]:
print(len(idx))
print(len(idx)/cs)
len(xs), xs[0].shape


600901
75112.625


(8, (75110,))

In [32]:
y = np.stack(c_out_dat[:-2])
y[:cs]


array([ 1, 33,  2, 72, 67, 73,  2, 68])

In [33]:
n_fac = 42


## Create and Train Model

In [34]:
def embedding_input(name, n_in, n_out):
    inp = Input(shape=(1,), dtype='int64', name=name+'_in')
    emb = Embedding(n_in, n_out, input_length=1, name=name+'_emb')(inp)
    return inp, Flatten()(emb)

In [35]:
c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]
n_hidden = 256
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax')

In [36]:
c_ins

[(c0_in, Reshape{2}.0),
 (c1_in, Reshape{2}.0),
 (c2_in, Reshape{2}.0),
 (c3_in, Reshape{2}.0),
 (c4_in, Reshape{2}.0),
 (c5_in, Reshape{2}.0),
 (c6_in, Reshape{2}.0),
 (c7_in, Reshape{2}.0)]

In [37]:
hidden = dense_in(c_ins[0][1])


In [38]:
for i in range(1,cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden])

In [39]:
c_out = dense_out(hidden)


In [40]:
model = Model([c[0] for c in c_ins], c_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [41]:
model.fit(xs, y, batch_size=64, nb_epoch=12, verbose=2)


Epoch 1/12
8s - loss: 2.5371
Epoch 2/12
9s - loss: 2.2517
Epoch 3/12
10s - loss: 2.1489
Epoch 4/12
9s - loss: 2.0780
Epoch 5/12
10s - loss: 2.0198
Epoch 6/12
11s - loss: 1.9722
Epoch 7/12
10s - loss: 1.9290
Epoch 8/12
10s - loss: 1.8912
Epoch 9/12
11s - loss: 1.8575
Epoch 10/12
11s - loss: 1.8264
Epoch 11/12
11s - loss: 1.7984
Epoch 12/12
11s - loss: 1.7715


In [42]:
def get_next(inp):
    idxs = [np.array(char_indices[c])[np.newaxis] for c in inp]
    p = model.predict(idxs)
    return chars[np.argmax(p)]

In [43]:
print(get_next('for thos'))
print(get_next('part of '))
print(get_next('queens a'))

 
t
n


---
## Returning Sequences 
---

Generate all outputs from 1 to cs

In [44]:
#c_in_dat = [[idx[i+n] for i in xrange(0, len(idx)-1-cs, cs)]
#            for n in range(cs)]
c_out_dat = [[idx[i+n] for i in xrange(1, len(idx)-cs, cs)]
            for n in range(cs)]

#notice the difference between the second RNN
#c_out_dat = [idx[i+cs] for i in xrange(0, len(idx)-1-cs, cs)]


In [45]:
xs = [np.stack(c[:-2]) for c in c_in_dat]
ys = [np.stack(c[:-2]) for c in c_out_dat]


In [46]:
[xs[n][:cs] for n in range(cs)]

[array([40,  1, 33,  2, 72, 67, 73,  2]),
 array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67])]

In [47]:
[ys[n][:cs] for n in range(cs)]
#notice that each y is x+1 position

[array([42,  1, 38, 44,  2,  9, 61, 73]),
 array([29, 43, 31, 71, 54,  9, 58, 61]),
 array([30, 45,  2, 74,  2, 76, 67, 58]),
 array([25, 40, 73, 73, 76, 61, 24, 71]),
 array([27, 40, 61, 61, 68, 54,  2, 58]),
 array([29, 39, 54,  2, 66, 73, 33,  2]),
 array([ 1, 43, 73, 62, 54,  2, 72, 67]),
 array([ 1, 33,  2, 72, 67, 73,  2, 68])]

## Training model


In [48]:
dense_in = Dense(n_hidden, activation='relu')
dense_hidden = Dense(n_hidden, activation='relu', init='identity')
dense_out = Dense(vocab_size, activation='softmax', name='output')

In [49]:
#Starting point with all zeroes 
inp1 = Input(shape=(n_fac,), name='zeros')
hidden = dense_in(inp1)

In [50]:
c_ins
#recall that c_ins = [embedding_input('c'+str(n), vocab_size, n_fac) for n in range(cs)]

[(c0_in, Reshape{2}.0),
 (c1_in, Reshape{2}.0),
 (c2_in, Reshape{2}.0),
 (c3_in, Reshape{2}.0),
 (c4_in, Reshape{2}.0),
 (c5_in, Reshape{2}.0),
 (c6_in, Reshape{2}.0),
 (c7_in, Reshape{2}.0)]

In [51]:
outs = []
for i in range(cs):
    c_dense = dense_in(c_ins[i][1])
    hidden = dense_hidden(hidden)
    hidden = merge([c_dense, hidden], mode='sum')
    # every layer now has an output
    outs.append(dense_out(hidden))

In [52]:
model = Model([inp1] + [c[0] for c in c_ins], outs)
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [53]:
zeros = np.tile(np.zeros(n_fac), (len(xs[0]),1))
zeros.shape

(75110, 42)

In [54]:
model.fit([zeros]+xs, ys, batch_size=64, nb_epoch=12, verbose= 2)


Epoch 1/12
13s - loss: 20.0531 - output_loss_1: 2.7000 - output_loss_2: 2.5601 - output_loss_3: 2.5108 - output_loss_4: 2.4680 - output_loss_5: 2.4642 - output_loss_6: 2.4502 - output_loss_7: 2.4578 - output_loss_8: 2.4420
Epoch 2/12
16s - loss: 17.8565 - output_loss_1: 2.5114 - output_loss_2: 2.3527 - output_loss_3: 2.2358 - output_loss_4: 2.1711 - output_loss_5: 2.1573 - output_loss_6: 2.1415 - output_loss_7: 2.1546 - output_loss_8: 2.1321
Epoch 3/12
17s - loss: 17.2702 - output_loss_1: 2.4968 - output_loss_2: 2.3300 - output_loss_3: 2.1779 - output_loss_4: 2.0868 - output_loss_5: 2.0593 - output_loss_6: 2.0389 - output_loss_7: 2.0515 - output_loss_8: 2.0291
Epoch 4/12
19s - loss: 16.9010 - output_loss_1: 2.4897 - output_loss_2: 2.3210 - output_loss_3: 2.1472 - output_loss_4: 2.0350 - output_loss_5: 1.9953 - output_loss_6: 1.9709 - output_loss_7: 1.9825 - output_loss_8: 1.9595
Epoch 5/12
26s - loss: 16.6424 - output_loss_1: 2.4858 - output_loss_2: 2.3171 - output_loss_3: 2.1263 - out

In [55]:
def get_nexts(inp):
    idxs = [char_indices[c] for c in inp]
    arrs = [np.array(i)[np.newaxis] for i in idxs]
    p = model.predict([np.zeros(n_fac)[np.newaxis,:]] + arrs)
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [56]:
get_nexts(' this is')


[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 't', ' ', 'c', 'n', ' ']

In [57]:
get_nexts(' part of')


[' ', 'p', 'a', 'r', 't', ' ', 'o', 'f']


['t', 'o', 'r', 't', ' ', 'o', 'f', ' ']

## Sequence model with keras 

To convert our previous keras model into a sequence model, simply add the 'return_sequences=True' parameter, and add TimeDistributed() around our dense layer.

In [58]:
n_hidden, n_fac, cs, vocab_size


(256, 42, 8, 86)

In [59]:
xs = [np.stack(c[:-2]) for c in c_in_dat]
ys = [np.stack(c[:-2]) for c in c_out_dat]

In [60]:
model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs),
        SimpleRNN(n_hidden, return_sequences=True, activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [61]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())


In [62]:
xs[0].shape


(75110,)

In [63]:
x_rnn=np.stack(xs, axis=1)
y_rnn=np.expand_dims(np.stack(ys, axis=1), -1)

In [64]:
x_rnn.shape, y_rnn.shape


((75110, 8), (75110, 8, 1))

In [65]:
model.fit(x_rnn, y_rnn, batch_size=64, nb_epoch=8, verbose=2)


Epoch 1/8
17s - loss: 2.4327
Epoch 2/8
16s - loss: 2.0037
Epoch 3/8
16s - loss: 1.8883
Epoch 4/8
16s - loss: 1.8267
Epoch 5/8
16s - loss: 1.7870
Epoch 6/8
16s - loss: 1.7599
Epoch 7/8
16s - loss: 1.7390
Epoch 8/8
15s - loss: 1.7229


In [66]:
def get_nexts_keras(inp):
    idxs = [char_indices[c] for c in inp]
    arr = np.array(idxs)[np.newaxis,:]
    p = model.predict(arr)[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [67]:
get_nexts_keras(' this is')


[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 'n', ' ', 'a', 'n', ' ']

## One - hot sequence with Keras 

will be replicated with Theano

In [68]:
model=Sequential([
        SimpleRNN(n_hidden, return_sequences=True, input_shape=(cs, vocab_size),
                  activation='relu', inner_init='identity'),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])
model.compile(loss='categorical_crossentropy', optimizer=Adam())

In [69]:
oh_ys = [to_categorical(o, vocab_size) for o in ys]
oh_y_rnn=np.stack(oh_ys, axis=1)

oh_xs = [to_categorical(o, vocab_size) for o in xs]
oh_x_rnn=np.stack(oh_xs, axis=1)

oh_x_rnn.shape, oh_y_rnn.shape

((75110, 8, 86), (75110, 8, 86))

In [70]:
model.fit(oh_x_rnn, oh_y_rnn, batch_size=64, nb_epoch=8, verbose = 2)

Epoch 1/8
16s - loss: 2.4542
Epoch 2/8
19s - loss: 2.0429
Epoch 3/8
18s - loss: 1.9266
Epoch 4/8
20s - loss: 1.8597
Epoch 5/8
20s - loss: 1.8157
Epoch 6/8
20s - loss: 1.7846
Epoch 7/8
19s - loss: 1.7611
Epoch 8/8
17s - loss: 1.7417


In [71]:
def get_nexts_oh(inp):
    idxs = np.array([char_indices[c] for c in inp])
    arr = to_categorical(idxs, vocab_size)
    p = model.predict(arr[np.newaxis,:])[0]
    print(list(inp))
    return [chars[np.argmax(o)] for o in p]

In [72]:
get_nexts_oh(' this is')


[' ', 't', 'h', 'i', 's', ' ', 'i', 's']


['t', 'h', 'e', 's', ' ', 'd', 's', ' ']

## Stateful model with Keras 

- LSTM

In [73]:
bs = 64 

model=Sequential([
        Embedding(vocab_size, n_fac, input_length=cs, batch_input_shape=(bs,8)),
        BatchNormalization(),
        LSTM(n_hidden, return_sequences=True, stateful=True),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
    ])

In [74]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())


In [75]:
#Since we're using a fixed batch shape, we have to ensure our inputs and outputs are a even multiple of the batch size.
mx = len(x_rnn)//bs*bs


In [82]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False, verbose=2)


INFO (theano.gof.compilelock): Refreshing lock /Users/lowyix/.theano/compiledir_Darwin-16.4.0-x86_64-i386-64bit-i386-2.7.11-64/lock_dir/lock
INFO (theano.gof.compilelock): Refreshing lock /Users/lowyix/.theano/compiledir_Darwin-16.4.0-x86_64-i386-64bit-i386-2.7.11-64/lock_dir/lock


Epoch 1/4
70s - loss: 2.2087
Epoch 2/4
84s - loss: 1.9673
Epoch 3/4
77s - loss: 1.8942
Epoch 4/4
98s - loss: 1.8501


In [83]:
model.optimizer.lr=1e-4
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False, verbose =2)


Epoch 1/4
84s - loss: 1.8178
Epoch 2/4
87s - loss: 1.7923
Epoch 3/4
82s - loss: 1.7709
Epoch 4/4
77s - loss: 1.7521


In [84]:
model.fit(x_rnn[:mx], y_rnn[:mx], batch_size=bs, nb_epoch=4, shuffle=False, verbose =2)


Epoch 1/4
52s - loss: 1.7352
Epoch 2/4
54s - loss: 1.7199
Epoch 3/4
56s - loss: 1.7056
Epoch 4/4
64s - loss: 1.6924
